# Aula 5: SQL - Exercícios

### Case Olist

A Olist é uma loja de departamentos que participa dos principais marketplaces do país:  Mercado Livre, Walmart.com, Americanas.com, Submarino, Casas Bahia entre outros. Mais informações podem ser encontradas [aqui](https://blog.olist.com/olist-chega-para-facilitar-a-vida-de-quem-quer-vender-na-internet-e-em-grandes-varejistas/).

![](https://ensinandomaquinasblog.files.wordpress.com/2020/11/f1.png?w=1024)
![](https://i.imgur.com/Jory0O3.png)

Abaixo, segue uma descrição de cada uma das tabelas contidas no arquivo `salesOlist.db` e em qual arquivo as informações se encontram. Também podem ser conferidas as informações sobre cada coluna.
 - **Customers**: guarda a cidade, o CEP e o estado do consumidor. Chave: customer_id.
  - Colunas:
    - **customer_id:** cada pedido possui um único `customer_id`;
    - **customer_unique_id:** identificador único de cada consumidor;
    - **customer_zip_code_prefix:** primeiros 5 dígitos do CEP do consumidor;
    - **customer_city:** cidade onde o consumidor se encontra; 
    - **customer_state:** estado onde o consumidor se encontra.
 - **Geolocation**: guarda informações de cidades tais como o CEP, latitude, longitude, nome e estado;
  - Colunas:
    - **geolocation_zip_code_prefix:** 5 primeiros dígitos do local;
    - **geolocation_lat:** latitude do local;
    - **geolocation_lng:** longitude do local;
    - **geolocation_city:** cidade onde se encontra o local; e
    - **geolocation_state:** estado onde se encontra o local.
 - **OrderItems**: guarda informações sobre quais produtos foram vendidos em cada pedido, seu valor, seu vendedor, preço de venda e preço de frete.
  - Colunas:
    - **order_id:** identificador único dos pedidos;
    - **order_item_id:** identificador sequencial indicando a quantidade de produtos por pedido;
    - **product_id:** identificador único de cada produto;
    - **seller_id:** identificador único de cada vendedor;
    - **price:** preço cobrado por produto; e
    - **freight_value:** preço cobrado por frete.
 - **OrderPayments**: guarda informações sobre o pagamento, parcelas e o valor pago por cada pedido.
  - Colunas:
    - **order_id:** identificador único de cada pedido;
    - **payment_sequential**;
    - **payment_type:** método escolhido para o pagamento;
    - **payment_installments:** número de parcelas; e
    - **payment_value:** total pago.
 - **OrderReviews**: guarda informações sobre *reviews* do pedido.
  - Colunas:
    - **review_id:** identificador único de cada avaliação;
    - **order_id:** identificador único de cada pedido;
    - **review_score:** nota dada ao pedido;
    - **review_comment_title:** título do comentário;
    - **review_comment_message:** comentário sobre o pedido;
    - **review_creation_date:** data do envio do questionário de satisfação; e
    - **review_answer_timestamp:** data da resposta ao questionário de satisfação .
 - **Orders**: guarda informações sobre o qual o consumidor efetuou a compra, o *status* do pedido, data da compra, datas de entrega (estimada e efetiva)
  - Colunas:
    - **order_id:** identificador único de cada pedido;
    - **customer_id:**; 
    - **order_status:** chave para a tabela `Customers` (cada pedido tem uma única chave);
    - **order_purchase_timestamp:** *timestamp* para a data do pedido;
    - **order_approved_at:** *timestamp* para a aprovação do pagamento;
    - **order_delivered_carrier_date:** *timestamp* para a data de postagem do pedido;
    - **order_delivered_customer_date:** data da entrega do pedido; e
    - **order_estimated_delivery_date:** data estimada da entrega.
 - **Products**: guarda informações sobre sobre a categoria e tamanho dos produtos vendidos. 
  - Colunas:
    - **product_id:** identificador único de cada pedido;
    - **product_category_name:** nome da categoria do produto;
    - **product_name_lenght:** comprimento do nome do produto;
    - **product_description_lenght:** comprimento da descrição do produto;
    - **product_photos_qty:** quantidade de fotos do produto;
    - **product_weight_g:** massa do produto em gramas;
    - **product_length_cm:** comprimento do produto em cm;
    - **product_height_cm:** altu e
    - product_width_cm.
 - **Sellers**: guarda informações do vendedor, tais como a cidade, estado e o código postal
  - Colunas:
    -**seller_id:** identificador único do vendedor;
    - **seller_zip_code_prefix:** guarda informações dos 4 primeiros dígitos do CEP do vendedor;
    - **seller_city:** cidade do vendedor; e
    - **seller_state:** estado do vendedor.
 - **QualifiedLeads**: informações de *leads* da Olist.
  - Colunas:
    - **mql_id:** identificador único da *lead*;
    - **first_contact_date:** data do primeiro contato;
    - **landing_page_id:** página que adquiriu a *lead*; e 
    - **origin:** tipo da mídia que adquriu a *lead*.
 - **ClosedDeals**: informações de negócios com vendedores 
    - **mql_id:** identificador único da lead;
    - **seller_id:** identificador único do vendedor;
    - **sdr_id:** identificador único do representante de vendas e desenvolvimento;
    - **sr_id:** identificador único do representante de vendas;
    - **won_date:** data do fechamento do contrato;
    - **business_segment:** segmento do *lead*; 
    - **lead_type:** tipo da *lead* (*online*, *offline*, indústria, etc);
    - **lead_behaviour_profile:** 
    - **has_company:**: o *lead* tem uma companhia (documentada)?
    - **has_gtin:** a companhia possi código de barras internacional para seus produtos?
    - **average_stock:** tamanho do estoque declarado;
    - **business_type:** tipo de negócio (revendedor/fabricante/etc);
    - **declared_product_catalog_size:** tamanho do catálogo declarado; e
    - **declared_monthly_revenue:** renda mensal declarada.

In [2]:
import sqlalchemy
import pandas as pd
import sqlite3
from pandasql import sqldf

In [3]:
con = sqlite3.connect('salesOlist.db')
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table" ORDER BY name')
cursor.fetchall()

[('ClosedDeals',),
 ('Customers',),
 ('Geolocation',),
 ('OrderItems',),
 ('OrderPayments',),
 ('OrderReviews',),
 ('Orders',),
 ('Products',),
 ('QualifiedLeads',),
 ('Sellers',)]

### Crie uma tabela com cada vendedor e o somatório que cada vendedor realizou (utilize a tabela `orderitems`). Em seguida, use junte essa tabela com a tabela de vendedores e ordene de forma decrescente pelo somatório que cada vendedor realizou

In [17]:
query = '''
SELECT *
FROM Sellers AS a
JOIN (
    SELECT seller_id
            ,SUM(price) AS valor_vendido_total
    FROM OrderItems
    GROUP BY seller_id
) AS b
    ON a.seller_id = b.seller_id
ORDER BY valor_vendido_total DESC
'''
pd.read_sql(query, con)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_id,valor_vendido_total
0,4869f7a5dfa277a7dca6462dcf3b52b2,14840,guariba,SP,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63
1,53243585a1d6dc2643021fd1853d8905,42738,lauro de freitas,BA,53243585a1d6dc2643021fd1853d8905,222776.05
2,4a3ca9315b744ce9f8e9374361493884,14940,ibitinga,SP,4a3ca9315b744ce9f8e9374361493884,200472.92
3,fa1c13f2614d7b5c4749cbc52fecda94,13170,sumare,SP,fa1c13f2614d7b5c4749cbc52fecda94,194042.03
4,7c67e1448b00f6e969d365cea6b010ab,08577,itaquaquecetuba,SP,7c67e1448b00f6e969d365cea6b010ab,187923.89
...,...,...,...,...,...,...
3090,34aefe746cd81b7f3b23253ea28bef39,81210,curitiba,PR,34aefe746cd81b7f3b23253ea28bef39,8.00
3091,702835e4b785b67a084280efca355756,36046,juiz de fora,MG,702835e4b785b67a084280efca355756,7.60
3092,1fa2d3def6adfa70e58c276bb64fe5bb,04106,sao paulo,SP,1fa2d3def6adfa70e58c276bb64fe5bb,6.90
3093,77128dec4bec4878c37ab7d6169d6f26,02610,sao paulo,SP,77128dec4bec4878c37ab7d6169d6f26,6.50


### Crie uma tabela com cada vendedor e a média que cada vendedor realizou (utilize a tabela `orderitems`) e filtre somente os vendedores que obtiveram média acima de R\$1000. Em seguida, use junte essa tabela com a tabela de vendedores e ordene de forma decrescente pela média que cada vendedor realizou

In [23]:
query = '''
SELECT *
FROM Sellers AS a
JOIN (
    SELECT seller_id
            ,AVG(price) AS valor_vendido_medio
    FROM OrderItems
    GROUP BY seller_id
    HAVING valor_vendido_medio > 1000
) AS b
    ON a.seller_id = b.seller_id
ORDER BY valor_vendido_medio DESC
'''
pd.read_sql(query, con)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_id,valor_vendido_medio
0,80ceebb4ee9b31afb6c6a916a574a1e2,86026,londrina,PR,80ceebb4ee9b31afb6c6a916a574a1e2,6729.000000
1,ee27a8f15b1dded4d213a468ba4eb391,74210,goiania,GO,ee27a8f15b1dded4d213a468ba4eb391,6499.000000
2,585175ec331ea177fa47199e39a6170a,39400,montes claros,MG,585175ec331ea177fa47199e39a6170a,3549.000000
3,abe021b01ba992245271b9aa422032df,80420,curitiba,PR,abe021b01ba992245271b9aa422032df,3360.000000
4,a00824eb9093d40e589b940ec45c4eb0,18590,bofete,SP,a00824eb9093d40e589b940ec45c4eb0,3133.323333
...,...,...,...,...,...,...
60,c88f62b4c386a59281014d677864d016,86707,arapongas,PR,c88f62b4c386a59281014d677864d016,1099.000000
61,ff69aa92bb6b1bf9b8b7a51c2ed9cf8b,08140,sao paulo,SP,ff69aa92bb6b1bf9b8b7a51c2ed9cf8b,1097.040000
62,612a743d294c27884fb7b80d2b19ba35,80620,curitiba,PR,612a743d294c27884fb7b80d2b19ba35,1019.233333
63,d6b664fa0667064c6d76394e7848d790,14801,araraquara,SP,d6b664fa0667064c6d76394e7848d790,1013.333333


### Avalie o Preco médio dos produtos e contagem de order_ids por Order_status e payment_type (realize um agrupamento por order status e payment type e veja qual é a forma de pagamento que as pessoas mais utilizam)

In [ ]:
query = '''
select
    order_status, 
    payment_type,
    count(order_id) qtd,
    avg(payment_value) media
from
(
select * from
    (select order_id, payment_type, payment_value
    from OrderPayments) as a
join
    (select order_id, order_status
    from Orders) as b
on a.order_id = b.order_id
)
group by order_status, payment_type
order by qtd desc
'''

In [31]:
query = '''
        SELECT a.order_status
                ,c.payment_type
                ,AVG(b.price)
                ,count(distinct(a.order_id))
        FROM Orders a
        JOIN OrderItems b
            ON a.order_id = b.order_id 
        JOIN OrderPayments c
            ON a.order_id = c.order_id
        GROUP BY a.order_status,c.payment_type
        ORDER BY a.order_status,c.payment_type
''' 
pd.read_sql(query, con)

,order_status,payment_type,AVG(b.price),count(distinct(a.order_id))
0,approved,credit_card,69.866667,2
1,canceled,boleto,145.395312,79
2,canceled,credit_card,179.976460,371
3,canceled,debit_card,420.021667,6
4,canceled,voucher,226.648621,21
5,delivered,boleto,104.186014,19191
6,delivered,credit_card,125.722126,74304
7,delivered,debit_card,107.615689,1485
8,delivered,voucher,103.334815,3679
9,invoiced,boleto,143.224945,66


### Veja quais são as categorias mais bem avaliadas pelos clientes (crie uma tabela com o nome da categoria do produto, a contagem de product_ids e a média de review_score e ordene a tabela dos mais bem avaliados na média para os piores avaliados)

In [27]:
query = '''
        SELECT d.product_category_name
                ,count(d.product_id)
                ,AVG(review_score)
                
        FROM OrderReviews a
        JOIN Orders b
           ON a.order_id = b.order_id
        JOIN OrderItems c
           ON a.order_id = c.order_id
        JOIN Products d
           ON c.product_id = d.product_id
               
        GROUP BY d.product_category_name               
        ORDER BY AVG(review_score) DESC
''' 

pd.read_sql(query, con)

,product_category_name,count(d.product_id),AVG(review_score)
0,cds_dvds_musicais,14,4.642857
1,fashion_roupa_infanto_juvenil,8,4.500000
2,livros_interesse_geral,553,4.439421
3,livros_importados,60,4.400000
4,construcao_ferramentas_ferramentas,103,4.359223
...,...,...,...
69,casa_conforto_2,30,3.366667
70,pc_gamer,9,3.333333
71,portateis_cozinha_e_preparadores_de_alimentos,15,3.266667
72,fraldas_higiene,39,3.256410


#### Importando toy datasets

In [ ]:
!pip install pycaret

In [18]:
from pycaret.datasets import get_data

In [19]:
get_data('index')

,Dataset,Data Types,Default Task,Target Variable 1,Target Variable 2,# Instances,# Attributes,Missing Values
0,anomaly,Multivariate,Anomaly Detection,None,None,1000,10,N
1,france,Multivariate,Association Rule Mining,InvoiceNo,Description,8557,8,N
2,germany,Multivariate,Association Rule Mining,InvoiceNo,Description,9495,8,N
3,bank,Multivariate,Classification (Binary),deposit,None,45211,17,N
4,blood,Multivariate,Classification (Binary),Class,None,748,5,N
5,cancer,Multivariate,Classification (Binary),Class,None,683,10,N
6,credit,Multivariate,Classification (Binary),default,None,24000,24,N
7,diabetes,Multivariate,Classification (Binary),Class variable,None,768,9,N
8,electrical_grid,Multivariate,Classification (Binary),stabf,None,10000,14,N
9,employee,Multivariate,Classification (Binary),left,None,14999,10,N


,Dataset,Data Types,Default Task,Target Variable 1,Target Variable 2,# Instances,# Attributes,Missing Values
0,anomaly,Multivariate,Anomaly Detection,None,None,1000,10,N
1,france,Multivariate,Association Rule Mining,InvoiceNo,Description,8557,8,N
2,germany,Multivariate,Association Rule Mining,InvoiceNo,Description,9495,8,N
3,bank,Multivariate,Classification (Binary),deposit,None,45211,17,N
4,blood,Multivariate,Classification (Binary),Class,None,748,5,N
5,cancer,Multivariate,Classification (Binary),Class,None,683,10,N
6,credit,Multivariate,Classification (Binary),default,None,24000,24,N
7,diabetes,Multivariate,Classification (Binary),Class variable,None,768,9,N
8,electrical_grid,Multivariate,Classification (Binary),stabf,None,10000,14,N
9,employee,Multivariate,Classification (Binary),left,None,14999,10,N


In [20]:
df = get_data('employee')

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary,left
0,0.38,0.53,2,157,3,0,0,sales,low,1
1,0.80,0.86,5,262,6,0,0,sales,medium,1
2,0.11,0.88,7,272,4,0,0,sales,medium,1
3,0.72,0.87,5,223,5,0,0,sales,low,1
4,0.37,0.52,2,159,3,0,0,sales,low,1


In [22]:
df['left'].value_counts()

0    11428
1     3571
Name: left, dtype: int64